In [7]:
import jsonlines
import os
import tqdm
import pandas as pd
import yaml

with open("config.yaml", "r") as fin:
    config = yaml.load(fin, Loader=yaml.FullLoader)

# Policy composition

In [8]:
df = {
    "policyA": [],
    "policyB": [],
    "choiceAB": [],
    "prompt": [],
    "summaryA": [],
    "summaryB": [],
    "batch": [],
    "split": []
}
input_files = [
    "comparisons_train.jsonl",
    "comparisons_valid.jsonl",
    "comparisons_test.jsonl"
]
for input_file in input_files:
    input_path = os.path.join(config["data_dir"], input_file)
    with jsonlines.open(input_path, "r") as fin:
        for line in tqdm.tqdm(fin):
            policy0 = line["example"]["summaries"][0]["policy"]
            policy1 = line["example"]["summaries"][1]["policy"]
            choice = line["choice"]
            assert line["example"]["summaries"][0]["text"].strip() == line["completion0"].strip()
            assert line["example"]["summaries"][1]["text"].strip() == line["completion1"].strip()
            if policy0 < policy1:
                policyA = policy0
                policyB = policy1
                choiceAB = line["choice"]
                summaryA = line["example"]["summaries"][0]["text"].strip()
                summaryB = line["example"]["summaries"][1]["text"].strip()
            else:
                policyA = policy1
                policyB = policy0
                choiceAB = 1 - line["choice"]
                summaryA = line["example"]["summaries"][1]["text"].strip()
                summaryB = line["example"]["summaries"][0]["text"].strip()
            df["policyA"].append(policyA)
            df["policyB"].append(policyB)
            df["choiceAB"].append(choiceAB)
            df["prompt"].append(line["prompt"])
            df["summaryA"].append(summaryA)
            df["summaryB"].append(summaryB)
            df["batch"].append(line["example"]["batch"])
            df["split"].append(line["example"]["split"])
df = pd.DataFrame(df)
df["n"] = 1

92858it [00:02, 41339.75it/s]
33082it [00:00, 39879.74it/s]
50715it [00:01, 40977.06it/s]


In [10]:
d = df.groupby(["policyA", "policyB"]).mean().reset_index().sort_values("choiceAB", ascending=False)
d = d[["policyA", "policyB", "choiceAB"]]
d2 = df.groupby(["policyA", "policyB"]).count().reset_index().sort_values("n", ascending=False)
d2 = d2[["policyA", "policyB", "n"]]
d = pd.merge(d, d2, on=["policyA", "policyB"])
d = d.sort_values("n", ascending=False)

In [12]:
d[0:20]

,policyA,policyB,choiceAB,n
216,ref,sup2,0.263811,18608
121,sup2,sup2,0.491735,16273
112,sup4_ppo_rm3_kl20,sup4_ppo_rm3_kl20,0.503437,8438
128,sup4_ppo_rm3_kl10,sup4_ppo_rm3_kl10,0.480788,8276
229,ref,sup1,0.189002,7947
111,sup1,sup1,0.506337,7653
115,sup4_6b_t0.7,sup4_6b_t0.7,0.500269,7442
224,ref,sup4_t0.7,0.228298,4919
104,sup4_t0.7,sup4_t0.7,0.520085,4257
80,sup4_6b_ppo_rm3_6b_kl15,sup4_6b_ppo_rm4_6b_kl14,0.567628,2669
